In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as LR
from matplotlib import pyplot as plt
from tools import *
from sklearn.decomposition import PCA
import plotly.express as px
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import PolynomialFeatures

# Get Data

In [2]:
def getObs(n):
    obs = {}
    for i in range(n):
        try:
            df = pd.read_csv(f'rocket-results/{i}.csv')
        except:
            print('Missing', i)
        obs[i] = df
    return obs

In [3]:
def initializeConfig():
    configs = {}
    with open('sample_list.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_100.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_200.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    with open('sample_list_300.pkl', 'rb') as f:
        d = pickle.load(f)
    configs.update(d)
    return configs

In [4]:
config = initializeConfig()
obs = getObs(400)

In [5]:
data = {}
for i in range(len(config)):
    data[i] = list(config[i]['S'].values()) + list(config[i]['B'].values())
    data[i].append(obs[i].max()['Altitude (ft)'])
    data[i].append(obs[i].mean()['Stability Margin (cal)'])
    data[i].append(obs[i].max()['Time (sec)'])
df = pd.DataFrame.from_dict(data, orient='index')
df.iloc[284]

/var/folders/ff/y3gj2gld0zqfjtq_lhxz03t00000gn/T/ipykernel_30053/2135078581.py:5: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data[i].append(obs[i].mean()['Stability Margin (cal)'])


0         3.734223
1         7.875106
2         2.509849
3         7.372139
4         8.753891
5         7.977354
6         7.539283
7         8.558976
8     32187.640000
9         5.071216
10       99.001530
Name: 284, dtype: float64

In [6]:
df = df.rename(columns={0: "Schord", 1: "Sspan", 2: "Ssweep", 3: "Stip", 4: "Bchord", 
                        5: "Bspan", 6: "Bsweep", 7: "Btip", 8: "Altitude", 9: "Stability", 10: "Time"})
df['Schordspan'] = df['Schord'] * df['Sspan']
df['Schordsweep'] = df['Schord'] * df['Ssweep']
df['Schordtip'] = df['Schord'] * df['Stip']
df['Sspansweep'] = df['Sspan'] * df['Ssweep']
df['SSpantip'] = df['Sspan'] * df['Stip']
df['SSweeptip'] = df['Ssweep'] * df['Stip']

df['Bchordspan'] = df['Bchord'] * df['Bspan']
df['Bchordsweep'] = df['Bchord'] * df['Bsweep']
df['Bchordtip'] = df['Bchord'] * df['Btip']
df['Bspansweep'] = df['Bspan'] * df['Bsweep']
df['BSpantip'] = df['Bspan'] * df['Btip']
df['BSweeptip'] = df['Bsweep'] * df['Btip']
alt = df.pop("Altitude")
stab = df.pop("Stability")
time = df.pop("Time")

df.insert(len(df.columns), "Altitude", alt)
df.insert(len(df.columns), "Stability", stab)
df.insert(len(df.columns), "Time", time)
df

,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip,Schordspan,Schordsweep,...,SSweeptip,Bchordspan,Bchordsweep,Bchordtip,Bspansweep,BSpantip,BSweeptip,Altitude,Stability,Time
0,9.465233,9.191411,6.313146,6.181568,4.253219,3.362766,2.004250,3.767645,86.998844,59.755399,...,39.025144,14.302581,8.524515,16.024619,6.739826,12.669711,7.551304,0.004903,-5.121752,0.0100
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,14.181241,36.623926,...,30.475799,49.826002,28.051867,48.398181,22.556595,38.917130,21.910210,77117.160000,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,45.139920,74.494323,...,69.178850,48.517110,42.439452,54.761021,53.116208,68.537590,59.951999,51216.000000,5.212006,125.0071
3,5.590219,9.233072,6.352605,8.429602,5.672564,5.705322,5.717624,3.420081,51.614896,35.512459,...,53.549937,32.363803,32.433584,19.400628,32.620884,19.512664,19.554736,0.004691,-2.489141,0.0100
4,2.916229,4.216994,4.711545,6.462613,4.887217,2.638681,9.605809,5.234709,12.297724,13.739946,...,30.448892,12.895807,46.945675,25.583158,25.346667,13.812727,50.283615,0.004818,-4.231226,0.0100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,8.729021,13.567499,...,47.680974,23.840776,49.293623,53.167952,30.650253,33.059270,68.353950,70830.750000,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,20.505433,43.245446,...,50.628851,40.195478,29.783881,48.611503,24.333390,39.715531,29.428252,72485.310000,1.614970,150.9995
397,7.364198,8.455093,4.617628,3.805883,4.280772,6.398586,8.552977,8.138172,62.264979,34.005123,...,17.574150,27.390888,36.613344,34.837657,54.726959,52.072793,69.605594,0.000000,-1.734951,0.0000
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,17.190049,20.815255,...,19.473850,16.587847,22.185683,24.114444,17.959727,19.521095,26.108803,73446.170000,0.853824,150.9995


In [7]:
# Interaction terms help marginally. 
X_train, X_test = df.iloc[:250, :-15], df.iloc[250:, :-15]

# Stability level 1: cluster into flight time of 20 and 90.
Y_train, Y_test = df.iloc[:250, -1] > 90, df.iloc[250:, -1] > 90

In [8]:
reg = LR()
reg.fit(X_train, Y_train)
sum(reg.predict(X_test) == Y_test)
print(sum(reg.predict(X_test.iloc[np.where(Y_test == False)]) == Y_test.iloc[np.where(Y_test == False)]) / sum(Y_test == False))
print(sum(reg.predict(X_test.iloc[np.where(Y_test == True)]) == Y_test.iloc[np.where(Y_test == True)]) / sum(Y_test == True))

0.958904109589041
0.8311688311688312


In [9]:
reg.predict_proba(X_test), Y_test

(array([[2.06790703e-01, 7.93209297e-01],
        [2.94485528e-02, 9.70551447e-01],
        [7.34116482e-02, 9.26588352e-01],
        [6.29730667e-01, 3.70269333e-01],
        [2.75170480e-02, 9.72482952e-01],
        [7.98236661e-01, 2.01763339e-01],
        [9.29147675e-01, 7.08523255e-02],
        [4.58622148e-01, 5.41377852e-01],
        [9.98272861e-01, 1.72713898e-03],
        [9.95574854e-01, 4.42514599e-03],
        [9.98992777e-01, 1.00722277e-03],
        [9.01672886e-01, 9.83271141e-02],
        [5.98911465e-01, 4.01088535e-01],
        [9.99862438e-01, 1.37562214e-04],
        [4.63388769e-02, 9.53661123e-01],
        [2.65241820e-01, 7.34758180e-01],
        [7.87214229e-02, 9.21278577e-01],
        [9.99388429e-01, 6.11570868e-04],
        [9.73115934e-01, 2.68840663e-02],
        [1.18782769e-01, 8.81217231e-01],
        [9.24417693e-01, 7.55823066e-02],
        [9.89045672e-01, 1.09543279e-02],
        [4.37253994e-01, 5.62746006e-01],
        [3.30376233e-02, 9.6696237

In [62]:
# Altitude
lr = LinearRegression()
filtered_df = df[df['Time'] > 90]


X_train, X_test = filtered_df.iloc[:150, :-3], filtered_df.iloc[150:, :-3]

# Stability level 1: cluster into flight time of 20 and 90.
Y_train, Y_test = filtered_df.iloc[:150, -3], filtered_df.iloc[150:, -3]


lr.fit(X_train, Y_train)
lr.score(X_train, Y_train)

0.9227099971500093

In [17]:
ridge = Ridge(alpha=10)
ridge.fit(X_train, Y_train)
lasso = Lasso(alpha=100)
lasso.fit(X_train, Y_train)

Lasso(alpha=100)

In [18]:
print("Training Errors (Lasso, Ridge, Normal)")
np.sqrt([MSE(Y_train, lasso.predict(X_train)), MSE(Y_train, ridge.predict(X_train)), MSE(Y_train, lr.predict(X_train))])

Training Errors (Lasso, Ridge, Normal)


array([4067.15206808, 4289.22133736, 4028.97177907])

In [36]:
print('Testing Errors (Lasso, Ridge, Normal)')
#np.sqrt([MSE(Y_test, lasso.predict(X_test)), MSE(Y_test, ridge.predict(X_test)), MSE(Y_test, lr.predict(X_test))])
raw.iloc[idxs]

Testing Errors (Lasso, Ridge, Normal)


,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip
212,5.259048,8.201739,7.418723,8.216527,2.640339,8.937041,9.653104,5.900797
213,5.556608,5.665979,2.141787,8.167383,6.534285,7.177163,3.075815,2.232603
214,7.736498,6.412773,6.981651,9.384999,7.258333,9.011802,2.222503,2.632106
215,6.513274,3.495698,8.171398,2.001211,8.004495,7.896190,6.976748,8.157834
217,9.498515,2.667657,2.510210,4.382181,4.978264,7.071330,5.616562,6.388213
...,...,...,...,...,...,...,...,...
91,6.334258,4.689458,8.331010,5.041502,3.633168,6.477348,7.138447,3.961190
94,5.745163,2.524135,4.400265,4.535364,5.006204,8.117043,2.555801,5.612034
95,8.769289,2.628346,6.056928,3.850910,8.635408,6.217285,8.914506,9.119260
96,6.756057,6.367212,6.210186,2.318735,9.237211,8.492059,7.481056,9.170537


In [71]:
poly = PolynomialFeatures(2)
raw = filtered_df.iloc[:,:8]
qr = LinearRegression()
lir = LinearRegression()
idxs = np.array(list(range(50,197)))# + list(range(0, 50)))

ridge = Ridge(alpha=0.6)
ridge.fit(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3])
lasso = Lasso(alpha=.2)
lasso.fit(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3])
lir.fit(raw.iloc[idxs], filtered_df.iloc[idxs,-3])
qr.fit(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3])
print('R2', qr.score(poly.fit_transform(raw.iloc[idxs]), filtered_df.iloc[idxs, -3]))
print('Test RMSE Quad', np.sqrt(MSE(filtered_df.iloc[150:, -3], qr.predict(poly.fit_transform(raw[150:])))))
print('Test RMSE ridge', np.sqrt(MSE(filtered_df.iloc[150:, -3], ridge.predict(poly.fit_transform(raw[150:])))))
print('Test RMSE lasso', np.sqrt(MSE(filtered_df.iloc[0:50, -3], lasso.predict(poly.fit_transform(raw[0:50])))))
print('Test RMSE Linear', np.sqrt(MSE(filtered_df.iloc[0:50, -3], lir.predict((raw[0:50])))))

print('Train RMSE', np.sqrt(MSE(filtered_df.iloc[idxs, -3], qr.predict(poly.fit_transform(raw.iloc[idxs])))))

R2 0.9802095842230094
Test RMSE Quad 2377.0002169894906
Test RMSE ridge 1230.9331987762987
Test RMSE lasso 2055.44657284463
Test RMSE Linear 4558.176077982448
Train RMSE 2126.7246957356047


/Users/alanyu/opt/miniconda3/envs/rocket/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.586e+07, tolerance: 3.360e+06
  model = cd_fast.enet_coordinate_descent(


In [21]:
# Should do this from a variety of inital points.
# What if negative ?
# Learning rate
def coordinateAscent(stab_classifier, alt_predictor, initial = [0,0,0,0,0,0,0,0], inc = 0.01, max_iter = 10000, confidence = 0.95):
    '''
    Keep increasing in direction until unstable. 
    '''
    
    x = np.array(initial)
    print('in', x)
    for _ in range(max_iter):
        for i in range(8):
            y = np.array(x)
            y[i] += np.sign(alt_predictor.coef_[i])*inc
            if 10 > y[i] > 3 and stab_classifier.predict_proba(poly.fit_transform([y]))[0,1] > confidence:
                x = np.array(y)
                break
        else:
            print('No more feasible directions at iteration', _, '.')
            break
    return x

In [22]:
filtered_df.iloc[138]

Schord             3.734223
Sspan              7.875106
Ssweep             2.509849
Stip               7.372139
Bchord             8.753891
Bspan              7.977354
Bsweep             7.539283
Btip               8.558976
Schordspan        29.407402
Schordsweep        9.372334
Schordtip         27.529212
Sspansweep        19.765324
SSpantip          58.056380
SSweeptip         18.502954
Bchordspan        69.832887
Bchordsweep       65.998065
Bchordtip         74.924351
Bspansweep        60.143526
BSpantip          68.277981
BSweeptip         64.528545
Altitude       32187.640000
Stability          5.071216
Time              99.001530
Name: 284, dtype: float64

In [503]:
np.array(samples.iloc[1])

array([4.85018295, 8.39532251, 4.51192896, 6.54405967, 9.2382256 ,
       8.89249209, 8.39590231, 9.20570753])

In [504]:
filtered_df.iloc[choices, :8], samples

(       Schord     Sspan    Ssweep      Stip    Bchord     Bspan    Bsweep  \
 395  2.608430  3.346465  5.201403  9.166944  6.192108  3.850187  7.960717   
 186  4.850183  8.395323  4.511929  6.544060  9.238226  8.892492  8.395902   
 16   5.565645  4.484935  9.236567  7.384820  7.277009  6.544632  2.931606   
 244  4.847887  8.631688  8.610444  2.181684  8.051872  5.556072  2.358074   
 196  6.254379  4.246962  3.976003  7.536317  2.002780  7.896346  2.316434   
 307  3.259887  4.166432  6.110471  9.050886  3.324642  7.487559  9.414183   
 200  3.385068  3.752941  9.547858  2.412589  4.683486  4.715880  3.874262   
 317  3.209788  5.616209  6.283975  7.220497  4.157478  5.305986  7.072609   
 348  4.278144  8.965851  7.574482  8.705158  4.564716  8.731888  8.793370   
 58   8.761546  5.687121  4.663327  9.913209  9.392404  6.941847  2.494271   
 
          Btip  
 395  8.586406  
 186  9.205708  
 16   9.798210  
 244  7.837840  
 196  8.834220  
 307  8.731582  
 200  6.656793  
 317

In [511]:
import warnings
warnings.filterwarnings("ignore")

#x = coordinateAscent(reg, lr, [4,5,8,8,6,8,7,9], confidence = 0.9)
#x, reg.predict_proba([x]), lr.predict([x])
choices = np.random.choice(len(filtered_df), 10)
samples = filtered_df.iloc[choices, :8]
print(samples)
for i in range(10):
    X0 = np.array(samples.iloc[i])
    #print('here1', X0)
    result = coordinateAscent(q_classifier, lr, X0, confidence = .6)
    #print('here2', X0)
    print(X0, qr.predict(poly.fit_transform([X0])), reg.predict_proba([X0])) #q_classifier.predict_proba(poly.fit_transform([x0])))
    print(qr.predict(poly.fit_transform([result])), result, reg.predict_proba([result]))
    print()
    

       Schord     Sspan    Ssweep      Stip    Bchord     Bspan    Bsweep  \
8    8.074672  3.479253  5.061755  2.201202  8.430403  6.003352  2.080965   
98   4.209668  4.827551  8.331124  6.053868  6.454260  7.426452  9.661657   
239  2.188132  2.929254  9.034666  7.787916  3.777422  6.617148  2.369873   
275  6.803003  4.441887  5.092325  4.497022  8.232802  6.858576  8.440981   
313  6.461930  2.461362  6.218536  5.257881  7.901328  4.562852  9.386697   
240  7.068569  2.215733  6.348256  7.886441  7.556871  9.588058  3.485536   
77   9.371178  8.915615  9.632276  3.661251  9.982313  9.721522  9.922950   
77   9.371178  8.915615  9.632276  3.661251  9.982313  9.721522  9.922950   
53   7.805058  2.542296  8.546093  4.073107  7.070326  7.273349  7.698943   
245  5.715487  7.102609  8.711010  8.826601  8.477324  7.753436  6.884072   

         Btip  
8    3.394916  
98   9.320130  
239  4.428782  
275  8.160722  
313  4.911477  
240  3.468907  
77   7.036977  
77   7.036977  
53   2.8

In [ ]:
reg.predict([[4,5,8,8,6,8,7,9]])

In [ ]:
np.sign(-3)

In [ ]:
df.max()

In [34]:
qr.coef_

array([-6.96594266e+14, -1.54643805e+03, -2.06355480e+04,  7.38034342e+03,
       -3.19684890e+02,  5.35090859e+02, -2.86035997e+03,  5.58915485e+02,
        1.96125182e+02,  5.07990159e+00,  2.07397768e+02, -7.94708498e+01,
        5.23132972e+01, -3.01971205e+00, -3.55805849e+00,  8.95297671e-01,
        7.29560568e+00,  8.90325206e+02, -1.85263007e+02,  3.30525145e+01,
        9.40290549e+01,  5.21366265e+02, -1.54500926e+02,  3.14041212e+01,
       -1.70052518e+02,  3.75519267e+01, -5.57080782e+01, -2.14698186e+02,
        1.28512416e+01, -5.56589464e+00, -6.04397563e+01,  5.55829158e+00,
        1.53778265e+01,  5.58794760e+00, -1.89868266e+01, -8.32512279e+00,
       -8.97972084e+01,  2.56453028e+01, -2.13957980e+01,  4.77570650e+01,
        6.00943454e+01,  3.44119778e+01, -3.43542879e+00, -3.07054111e+01,
       -3.01965488e+01])

In [397]:
def coordinateAscentQuad(stab_classifier, alt_predictor, initial = [0,0,0,0,0,0,0,0], inc = 0.0000001, max_iter = 10000, confidence = 0.95):
    '''
    Keep increasing in direction until unstable. 
    '''
    M = {}
    c = 9
    for i in range(8):
        for j in range(i, 8):
            M[(i,j)] = c
            c += 1
    def partials(x):
        grad = []
        coef = alt_predictor.coef_
        for i in range(8):
            g = coef[1 + i]
            for j in range(8):
                if i == j:
                    g += 2*x[i]*coef[M[i,i]]
                else:
                    g += coef[M[min([i,j]), max([i,j])]]*x[j]
            grad.append(g)
        return np.array(grad)
    
    x = initial
    old = qr.predict(poly.fit_transform([x]))
    for _ in range(max_iter):
        #for i in range(8):
        y = np.array(x)
        # y[i] += np.sign(alt_predictor.coef_[i])*inc
        # print(y, partials(x)*inc)
        y += partials(x) * inc
        if np.all(y > 2.5) and y[0] > 2.5 and y[4] > 2.5 and qr.predict(poly.fit_transform([y])) > old:
            nxt = stab_classifier.predict_proba(poly.fit_transform([y]))[0,1]
            if nxt > confidence or np.random.rand() > 0.5:
                if nxt <= confidence:
                    print('Non greedy step')
                x = y[:]
                #print(qr.predict(poly.fit_transform([y])) - old)
                old = qr.predict(poly.fit_transform([y]))
                continue
            else:
                break
        
        print('No more feasible directions at iteration', _, '.', qr.predict(poly.fit_transform([x])) - qr.predict(poly.fit_transform([initial])))
        break
    return x

In [367]:
x0 = [8.762542, 2.403241, 8.77518, 3.823229, 4.85104, 4.60577, 8.116536, 8.553734]
result = coordinateAscentQuad(reg, qr, x0, confidence = .6)
print(qr.predict(poly.fit_transform([x0])), reg.predict_proba([x0]))#q_classifier.predict_proba(poly.fit_transform([x0])))
qr.predict(poly.fit_transform([result])), result, q_classifier.predict_proba(poly.fit_transform([result]))

No more feasible directions at iteration 0 . [0.]
[87794.] [[0.33229918 0.66770082]]


(array([87794.]),
 [8.762542, 2.403241, 8.77518, 3.823229, 4.85104, 4.60577, 8.116536, 8.553734],
 array([[0.92912015, 0.07087985]]))

In [207]:
res = [8.722, 2.0002, 8.837, 3.825, 4.845, 4.5285, 8.1288, 8.54]
#qr.predict(poly.fit_transform([res])), result, q_classifier.predict_proba([res])
df.iloc[:350, :8]

,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip
0,9.465233,9.191411,6.313146,6.181568,4.253219,3.362766,2.004250,3.767645
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342
3,5.590219,9.233072,6.352605,8.429602,5.672564,5.705322,5.717624,3.420081
4,2.916229,4.216994,4.711545,6.462613,4.887217,2.638681,9.605809,5.234709
...,...,...,...,...,...,...,...,...
345,8.805210,2.507928,8.884587,9.715194,9.885078,3.430887,2.563985,7.895791
346,3.425300,3.120913,6.224504,3.867722,2.054994,5.288580,7.228844,3.771977
347,4.561526,5.292796,4.021472,4.625128,5.743694,4.882127,8.869142,2.461874
348,4.278144,8.965851,7.574482,8.705158,4.564716,8.731888,8.793370,7.012021


In [27]:
# q classifier disagrees a lot with linear. 
q_classifier = LR()
idxs = np.array(list(range(200, 400)) + list(range(100)))

X_train = poly.fit_transform(df.iloc[idxs, :8])
Y_train = df.iloc[idxs, -1] > 90

X_test = poly.fit_transform(df.iloc[100:200, :8])
Y_test = df.iloc[100:200, -1] > 90

q_classifier.fit(X_train, Y_train)

print(sum(q_classifier.predict(X_test) == Y_test))
print(sum(q_classifier.predict(X_test[np.where(Y_test == False)]) == Y_test.iloc[np.where(Y_test == False)]) / sum(Y_test == False))
print(sum(q_classifier.predict(X_test[np.where(Y_test == True)]) == Y_test.iloc[np.where(Y_test == True)]) / sum(Y_test == True))

93
0.9272727272727272
0.9333333333333333


/Users/alanyu/opt/miniconda3/envs/rocket/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [396]:
samples = filtered_df.iloc[np.random.choice(len(filtered_df), 10), :8]
for i in range(10):
    X0 = np.array(samples.iloc[i])
    result = coordinateAscentQuad(q_classifier, qr, X0, confidence = .6)
    print(X0, qr.predict(poly.fit_transform([X0])), reg.predict_proba([X0])) #q_classifier.predict_proba(poly.fit_transform([x0])))
    print(qr.predict(poly.fit_transform([result])), result, reg.predict_proba([result]))
    print()
    

No more feasible directions at iteration 6 . [114.625]
[8.80520968 2.50792814 8.8845873  9.7151944  9.88507815 3.43088659
 2.56398478 7.8957911 ] [84950.375] [[0.18172691 0.81827309]]
[85065.] [8.80453914 2.50006519 8.88594417 9.71480747 9.88491451 3.4287996
 2.5643135  7.89546474] [[0.18133645 0.81866355]]

No more feasible directions at iteration 0 . [0.]
[5.49215417 8.26276766 6.21403859 8.44036831 9.6772564  6.78353181
 2.86848029 5.2141351 ] [35755.875] [[0.31939203 0.68060797]]
[35755.875] [5.49215417 8.26276766 6.21403859 8.44036831 9.6772564  6.78353181
 2.86848029 5.2141351 ] [[0.31939203 0.68060797]]

No more feasible directions at iteration 0 . [0.]
[3.66606175 6.9471738  5.86707696 7.66763302 6.5814646  7.81747032
 9.11547434 4.66208075] [39312.25] [[0.31995839 0.68004161]]
[39312.25] [3.66606175 6.9471738  5.86707696 7.66763302 6.5814646  7.81747032
 9.11547434 4.66208075] [[0.31995839 0.68004161]]

No more feasible directions at iteration 0 . [0.]
[2.22830359 9.13134253 2

In [195]:
np.array(samples.iloc[0])

array([4.73444547e+00, 4.55713543e+00, 4.76768044e+00, 3.43126218e+00,
       6.18119505e+00, 7.97143628e+00, 6.97046524e+00, 6.40215195e+00,
       2.15755092e+01, 2.25723231e+01, 1.62451237e+01, 2.17269654e+01,
       1.56367264e+01, 1.63591616e+01, 4.92730025e+01, 4.30858052e+01,
       3.95729500e+01, 5.55646194e+01, 5.10343463e+01, 4.46259776e+01,
       5.08914500e+04, 3.79841953e+00, 1.24006900e+02])

In [233]:
# linear classifier is not very good ?

In [398]:
from scipy.optimize import LinearConstraint, NonlinearConstraint, Bounds, minimize
bounds = Bounds([[2.5, np.inf], [2.5, np.inf], [2.5, np.inf]])#, [2.5, np.inf])#, [2.5, np.inf], [2.5, np.inf], [2.5, np.inf])
def stab_constraint(X):
    return q_classifier.predict_proba(poly.fit_transform([X]))[0,1]-0.7
def l0(X):
    return X[0]-7.5
def l1(X):
    return X[1]-3.5
def l2(X):
    return X[2]-3.5
def l3(X):
    return X[3]-3.5
def l4(X):
    return X[4]-7.5
def l5(X):
    return X[5]-3.5
def l6(X):
    return X[6]-3.5
def l7(X):
    return X[7]-3.5

def u0(X):
    return 10-X[0]
def u1(X):
    return 10-X[1]
def u2(X):
    return 10-X[2]
def u3(X):
    return 10-X[3]
def u4(X):
    return 10-X[4]
def u5(X):
    return 10-X[5]
def u6(X):
    return 10-X[6]
def u7(X):
    return 10-X[7]
def objective(X):
    return -qr.predict(poly.fit_transform([X]))[0]
constraint = NonlinearConstraint(stab_constraint, 0.6, np.inf)

In [399]:
ineq_cons = {'type': 'ineq',
             'fun' : stab_constraint}
L0 = {'type': 'ineq',
             'fun' : l0}
L1 = {'type': 'ineq',
             'fun' : l1}
L2 = {'type': 'ineq',
             'fun' : l2}
L3 = {'type': 'ineq',
             'fun' : l3}
L4 = {'type': 'ineq',
             'fun' : l4}
L5 = {'type': 'ineq',
             'fun' : l5}
L6 = {'type': 'ineq',
             'fun' : l6}
L7 = {'type': 'ineq',
             'fun' : l7}
U0 = {'type': 'ineq',
             'fun' : u0}
U1 = {'type': 'ineq',
             'fun' : u1}
U2 = {'type': 'ineq',
             'fun' : u2}
U3 = {'type': 'ineq',
             'fun' : u3}
U4 = {'type': 'ineq',
             'fun' : u4}
U5 = {'type': 'ineq',
             'fun' : u5}
U6 = {'type': 'ineq',
             'fun' : u6}
U7 = {'type': 'ineq',
             'fun' : u7}
# x = minimize(objective, X0, method='trust-constr', options = {'disp':True}).x#, constraints = [ineq_cons], options={'ftol': 1e-9, 'disp': True})

In [400]:
print(X0)
res = minimize(objective, X0, method='COBYLA', 
               constraints = [ineq_cons, L0, L1, L2, L3, L4, L5, L6, L7, U0, U1, U2, U3, U4, U5, U6, U7], 
               options = {'verbose':1, 'display':1, 'maxiter':1000}, bounds = bounds)#,constraints = [constraint], options = {'verbose':1})#, bounds = bounds)
res

[4.31666012 3.86802621 2.32225016 2.61348146 6.03567236 4.31940999
 5.86434536 9.52098839]


     fun: -82346.125
   maxcv: 7.883169672595614e-10
 message: 'Optimization terminated successfully.'
    nfev: 172
  status: 1
 success: True
       x: array([ 7.5       ,  3.5       , 10.        ,  7.38765915, 10.        ,
        3.55253104, 10.        ,  3.5       ])

In [284]:
r = [3.25988046, 4.16648426, 6.11046753, 9.05087259, 3.32468452,
       7.48760038, 9.41419767, 8.73158409]

In [255]:
qr.predict(poly.fit_transform([r])), qr.predict(poly.fit_transform([X0]))

(array([58798.5]), array([58798.875]))

In [270]:
X0 = np.array([7.00933451, 6.45751096, 8.41870936, 7.93664641, 6.46337112, 8.07023648, 6.12427831, 5.59915807])
stab_constraint(X0)

0.999999999520929

In [300]:
objective(x)

-443163750.0

In [376]:
np.random.rand(1)

array([0.23559022])

In [401]:
qr

LinearRegression()

In [404]:
import torch
class Classifier(torch.nn.Module):

    def __init__(self):
        super(Classifier, self).__init__()

        self.linear1 = torch.nn.Linear(45, 90)
        self.activation1 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(90, 60)
        self.activation2 = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(60, 30)
        self.activation3 = torch.nn.ReLU()
        self.linear4 = torch.nn.Linear(30, 10)
        self.activation4 = torch.nn.ReLU()
        self.linear5 = torch.nn.Linear(10, 1)

    def forward(self, x):
        
        x = self.linear1(x)
        x = self.activation1(x)
        x = self.linear2(x)
        x = self.activation2(x)
        x = self.linear3(x)
        x = self.activation3(x)
        x = self.linear4(x)
        x = self.activation4(x)
        x = self.linear5(x)
        
        return torch.sigmoid(x)

In [405]:
classif = Classifier()
classif.double()

Classifier(
  (linear1): Linear(in_features=45, out_features=90, bias=True)
  (activation1): ReLU()
  (linear2): Linear(in_features=90, out_features=60, bias=True)
  (activation2): ReLU()
  (linear3): Linear(in_features=60, out_features=30, bias=True)
  (activation3): ReLU()
  (linear4): Linear(in_features=30, out_features=10, bias=True)
  (activation4): ReLU()
  (linear5): Linear(in_features=10, out_features=1, bias=True)
)

In [412]:
from torch.utils import data
from torch import nn

def load_arrays(data_arrays, batch_size, train = True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle = train)

In [413]:
poly = PolynomialFeatures(2)
raw_X = df.iloc[:,:8]
raw_Y = df.iloc[:,-1] > 90
batch_size = 10

train_X = torch.from_numpy(poly.fit_transform(raw_X[:300])).double()
train_Y = torch.from_numpy(np.array([[x] for x in raw_Y[:300]])).double()

test_X = torch.from_numpy(poly.fit_transform(raw_X[300:])).double()
test_Y = torch.from_numpy(np.array([[x] for x in raw_Y[300:]])).double()
data_iter = load_arrays((train_X, train_Y), 10)

In [414]:
loss = nn.BCELoss()

In [415]:
trainer = torch.optim.SGD(classif.parameters(), lr=0.01)
num_epochs = 250
for epoch in range(num_epochs):
 
    for X, y in data_iter:
        X.requires_grad = True
        l = loss(classif(X) ,y)

        trainer.zero_grad() #sets gradients to zero

        l.backward() # back propagation

        trainer.step() # parameter update
        
    l = loss(classif(train_X), train_Y)
    if epoch % 50 == 0:
        print(f'epoch {epoch + 1}, loss {l:f}')
        #print(X.grad)

epoch 1, loss 0.433341
epoch 51, loss 0.068826
epoch 101, loss 0.015560
epoch 151, loss 0.009924
epoch 201, loss 0.008643


In [416]:
loss(classif(test_X), test_Y)

tensor(0.6437, dtype=torch.float64, grad_fn=<BinaryCrossEntropyBackward0>)

In [418]:
sum((classif(test_X) >= 0.5) == test_Y)/len(test_Y)
X0

array([4.31666012, 3.86802621, 2.32225016, 2.61348146, 6.03567236,
       4.31940999, 5.86434536, 9.52098839])

In [428]:
def coordinateAscentNN(stab_classifier, alt_predictor, initial = [0,0,0,0,0,0,0,0], inc = 0.0000001, max_iter = 10000, confidence = 0.95):
    '''
    Keep increasing in direction until unstable. 
    '''
    M = {}
    c = 9
    for i in range(8):
        for j in range(i, 8):
            M[(i,j)] = c
            c += 1
    def partials(x):
        grad = []
        coef = alt_predictor.coef_
        for i in range(8):
            g = coef[1 + i]
            for j in range(8):
                if i == j:
                    g += 2*x[i]*coef[M[i,i]]
                else:
                    g += coef[M[min([i,j]), max([i,j])]]*x[j]
            grad.append(g)
        return np.array(grad)
    
    x = initial
    old = qr.predict(poly.fit_transform([x]))
    for _ in range(max_iter):
        #for i in range(8):
        y = np.array(x)
        # y[i] += np.sign(alt_predictor.coef_[i])*inc
        # print(y, partials(x)*inc)
        y += partials(x) * inc
        if np.all(y > 2.5) and y[0] > 2.5 and y[4] > 2.5 and qr.predict(poly.fit_transform([y])) > old:
            nxt = stab_classifier(torch.from_numpy(poly.fit_transform([y])))[0]
            if nxt > confidence or np.random.rand() > 0.5:
                if nxt <= confidence:
                    print('Non greedy step')
                x = y[:]
                #print(qr.predict(poly.fit_transform([y])) - old)
                old = qr.predict(poly.fit_transform([y]))
                continue
            else:
                break
        
        print('No more feasible directions at iteration', _, '.', qr.predict(poly.fit_transform([x])) - qr.predict(poly.fit_transform([initial])))
        break
    return x

In [443]:
samples = filtered_df.iloc[np.random.choice(len(filtered_df), 197), :8]
best = 0
for i in range(197):
    X0 = np.array(samples.iloc[i])
    result = coordinateAscentNN(classif, qr, X0, confidence = .6)
    print(X0, qr.predict(poly.fit_transform([X0])), reg.predict_proba([X0])) #q_classifier.predict_proba(poly.fit_transform([x0])))
    print(qr.predict(poly.fit_transform([result])), result, reg.predict_proba([result]))
    print()
    best = max(best, qr.predict(poly.fit_transform([result])))
print('BEST', best)
    


No more feasible directions at iteration 0 . [0.]
[4.63909766 7.2714982  6.86832483 2.26021714 3.85604358 7.69309272
 2.95277771 3.86600877] [39688.] [[0.39876633 0.60123367]]
[39688.] [4.63909766 7.2714982  6.86832483 2.26021714 3.85604358 7.69309272
 2.95277771 3.86600877] [[0.39876633 0.60123367]]

No more feasible directions at iteration 0 . [0.]
[3.51324278 3.07721551 6.85920372 6.88783709 9.21928215 6.05656858
 5.77688879 2.11950605] [76448.125] [[0.11878277 0.88121723]]
[76448.125] [3.51324278 3.07721551 6.85920372 6.88783709 9.21928215 6.05656858
 5.77688879 2.11950605] [[0.11878277 0.88121723]]

No more feasible directions at iteration 4 . [1.125]
[4.24420862 9.87886603 9.85150662 9.96195105 6.38345437 6.98902191
 3.50112154 8.66333995] [36233.625] [[0.44212496 0.55787504]]
[36234.75] [4.24433101 9.87899204 9.85165741 9.96170812 6.38350568 6.98938831
 3.50092648 8.66324658] [[0.44202687 0.55797313]]

No more feasible directions at iteration 0 . [0.]
[3.44278251 2.09204241 3.67

No more feasible directions at iteration 4176 . [33443.625]
[3.03349873 5.98991279 7.60201447 2.65350567 6.81253481 9.09042745
 2.53521989 7.94049156] [47183.375] [[0.00537574 0.99462426]]
[80627.] [2.58751113 2.50036976 8.21152371 2.64911089 6.60990868 8.45690626
 2.71406409 7.88430465] [[0.00120486 0.99879514]]

[2.60843046 3.34646482 5.20140329 9.16694417 6.19210809 3.8501873
 7.96071749 8.58640565] [69371.625] [[0.6483941 0.3516059]]
[69371.625] [2.60843046 3.34646482 5.20140329 9.16694417 6.19210809 3.8501873
 7.96071749 8.58640565] [[0.6483941 0.3516059]]

No more feasible directions at iteration 1639 . [20820.125]
[3.071609   4.24563298 5.1245792  7.19735674 6.06970858 7.5685513
 4.64635842 9.32825589] [54772.125] [[0.0190335 0.9809665]]
[75592.25] [2.93512764 2.50042656 5.61908671 7.08058112 6.02669841 7.34566305
 4.71663821 9.25985778] [[0.00785209 0.99214791]]

No more feasible directions at iteration 2429 . [20719.875]
[7.39791924 4.56907301 3.3302831  8.08162675 7.59573274 

No more feasible directions at iteration 768 . [8237.375]
[9.22545751 3.26607377 8.6440655  9.16676316 4.04581057 9.74295102
 4.85377851 8.01883358] [63247.875] [[0.00181311 0.99818689]]
[71485.25] [9.14491063 2.50048979 8.73778704 9.13014529 3.99701522 9.59101834
 4.90659439 7.9992584 ] [[0.00134077 0.99865923]]

No more feasible directions at iteration 5967 . [42296.25]
[5.71548708 7.10260945 8.71101024 8.82660091 8.47732381 7.75343561
 6.88407201 6.98627806] [41477.875] [[0.07955794 0.92044206]]
[83774.125] [5.31972671 2.50087625 9.42643085 8.53055844 8.34281512 6.9512251
 7.07878708 6.76779235] [[0.01143352 0.98856648]]

[2.75783751 5.7397775  9.11131928 8.01095659 2.98231941 7.33500087
 9.95023045 9.04886334] [51908.125] [[0.2129533 0.7870467]]
[87443.] [ 2.51149826  2.73799765  9.61392318  7.8386911   2.93508022  7.0218149
 10.0093047   8.90867265] [[0.05214849 0.94785151]]

No more feasible directions at iteration 2 . [0.625]
[6.08199065 8.40358411 5.89640539 4.42759739 7.531558

Non greedy step
[4.20966833 4.82755058 8.33112372 6.05386772 6.45425962 7.42645223
 9.66165679 9.32013025] [57937.] [[0.04242566 0.95757434]]
[80969.75] [4.02767582 2.87234451 8.65391752 5.99019685 6.40756566 7.15124992
 9.72474386 9.22114327] [[0.01700131 0.98299869]]

Non greedy step
[3.15625135 5.03102599 4.70969599 3.06083945 9.29774736 4.79142794
 8.05243392 4.79614511] [52802.25] [[0.65031248 0.34968752]]
[52812.125] [3.15618255 5.0300745  4.71002007 3.06082923 9.29777457 4.79133385
 8.05245251 4.7961173 ] [[0.6501873 0.3498127]]

No more feasible directions at iteration 0 . [0.]
[8.40079326 9.3841775  2.31882667 5.22013186 6.65516209 8.69986095
 6.19268974 9.05128145] [31204.375] [[0.22412324 0.77587676]]
[31204.375] [8.40079326 9.3841775  2.31882667 5.22013186 6.65516209 8.69986095
 6.19268974 9.05128145] [[0.22412324 0.77587676]]

No more feasible directions at iteration 1036 . [14914.5]
[6.7654802  3.66965415 2.85853634 3.91202373 9.98809675 6.17321435
 9.28874301 5.78085487]

No more feasible directions at iteration 1036 . [14914.5]
[6.7654802  3.66965415 2.85853634 3.91202373 9.98809675 6.17321435
 9.28874301 5.78085487] [56080.125] [[0.14166532 0.85833468]]
[70994.625] [6.6750139  2.50085549 3.23173049 3.89844626 9.99429681 6.02664176
 9.34348515 5.74086299] [[0.08151386 0.91848614]]

No more feasible directions at iteration 53 . [764.125]
[5.77545084 2.56032368 2.84687668 5.7254832  8.17997311 7.81889402
 6.69193553 5.6801515 ] [64741.] [[0.01323533 0.98676467]]
[65505.125] [5.7707049  2.50052124 2.86604694 5.72323862 8.17912793 7.81106663
 6.69564954 5.67876571] [[0.01283602 0.98716398]]

No more feasible directions at iteration 0 . [0.]
[6.08523087 2.4149837  5.01563924 3.35185074 9.76801024 7.05517285
 6.78332714 7.67508825] [74143.] [[0.00788028 0.99211972]]
[74143.] [6.08523087 2.4149837  5.01563924 3.35185074 9.76801024 7.05517285
 6.78332714 7.67508825] [[0.00788028 0.99211972]]

No more feasible directions at iteration 2824 . [34906.125]
[3.21984

No more feasible directions at iteration 5907 . [44795.5]
[3.07421164 7.20075127 6.37380909 4.27023187 5.28944986 7.86059427
 6.30914519 7.51564167] [39819.75] [[0.17285526 0.82714474]]
[84615.25] [2.63884252 2.50018822 7.57694201 4.15281474 5.21459901 7.48489185
 6.42165652 7.38295921] [[0.01374018 0.98625982]]

No more feasible directions at iteration 351 . [200.5]
[9.09445845 7.44103934 4.08270257 5.52141517 8.45846964 9.00860641
 6.31020385 6.91349473] [35338.] [[0.04440207 0.95559793]]
[35538.5] [9.09602563 7.39264381 4.14327577 5.52064915 8.46090375 9.04003972
 6.31050215 6.90949568] [[0.04125168 0.95874832]]

No more feasible directions at iteration 0 . [0.]
[2.24938549 7.1355691  9.60357587 6.03064032 9.55679646 9.94656893
 5.29311738 3.4859656 ] [43081.75] [[0.00652788 0.99347212]]
[43081.75] [2.24938549 7.1355691  9.60357587 6.03064032 9.55679646 9.94656893
 5.29311738 3.4859656 ] [[0.00652788 0.99347212]]

No more feasible directions at iteration 1191 . [17686.875]
[3.471628

Non greedy step
Non greedy step
Non greedy step
Non greedy step
Non greedy step
[3.0885825  7.34552638 5.19000697 2.65042012 8.70100458 8.68577093
 7.85250398 8.18927875] [37615.625] [[0.03303762 0.96696238]]
[72686.125] [2.65860044 3.05978309 6.44782432 2.64323302 8.60356743 8.4692395
 7.99807322 7.99079919] [[0.00251886 0.99748114]]

No more feasible directions at iteration 0 . [0.]
[2.0492438  3.76322981 5.37982854 5.86245733 4.71079861 5.4551199
 6.22584348 9.5382352 ] [64807.625] [[0.23543077 0.76456923]]
[64807.625] [2.0492438  3.76322981 5.37982854 5.86245733 4.71079861 5.4551199
 6.22584348 9.5382352 ] [[0.23543077 0.76456923]]

No more feasible directions at iteration 9297 . [46140.5]
[5.25904766 8.20173937 7.41872267 8.21652658 2.64033946 8.93704144
 9.65310424 5.90079695] [37010.75] [[0.40766783 0.59233217]]
[83151.25] [4.85775492 2.50046812 8.69518449 7.8282839  2.5860981  9.06602292
 9.78695712 5.75279967] [[0.01275669 0.98724331]]

No more feasible directions at iteration

In [439]:
samples = filtered_df.iloc[np.random.choice(len(filtered_df), 197), :8]
best = 0
for i in range(197):
    X0 = np.array(samples.iloc[i])
    result = coordinateAscentNN(q_classifier, qr, X0, confidence = .6)
    print(X0, qr.predict(poly.fit_transform([X0])), reg.predict_proba([X0])) #q_classifier.predict_proba(poly.fit_transform([x0])))
    print(qr.predict(poly.fit_transform([result])), result, reg.predict_proba([result]))
    print()
    best = max(best, qr.predict(poly.fit_transform([result]))
print('BEST', best)

,Schord,Sspan,Ssweep,Stip,Bchord,Bspan,Bsweep,Btip,Schordspan,Schordsweep,...,SSweeptip,Bchordspan,Bchordsweep,Bchordtip,Bspansweep,BSpantip,BSweeptip,Altitude,Stability,Time
1,5.050903,2.807665,7.250966,4.202998,7.871765,6.329711,3.563606,6.148326,14.181241,36.623926,...,30.475799,49.826002,28.051867,48.398181,22.556595,38.917130,21.910210,77117.16,1.278655,154.9973
2,8.796076,5.131824,8.469040,8.168441,6.226139,7.792487,6.816335,8.795342,45.139920,74.494323,...,69.178850,48.517110,42.439452,54.761021,53.116208,68.537590,59.951999,51216.00,5.212006,125.0071
5,8.562160,4.337768,7.930397,4.372560,9.417980,8.246008,2.728282,6.326085,37.140668,67.901332,...,34.676133,77.660732,25.694900,59.578937,22.497430,52.164942,17.259340,54082.50,4.485508,129.0079
6,8.703811,2.188438,9.143090,2.635864,9.087634,8.094306,6.826608,3.754130,19.047753,79.579730,...,24.099946,73.558093,62.037714,34.116158,55.256655,30.387077,25.627973,78784.51,0.613785,156.9962
8,8.074672,3.479253,5.061755,2.201202,8.430403,6.003352,2.080965,3.394916,28.093824,40.872014,...,11.141944,50.610675,17.543371,28.620512,12.492762,20.380874,7.064700,59576.45,2.920013,136.0078
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,5.247843,7.799644,8.441005,9.658211,6.937770,7.882306,6.353957,9.196173,40.931306,44.297067,...,81.524999,54.685626,44.082290,63.800932,50.083829,72.487046,58.432082,39893.78,5.851534,111.0041
395,2.608430,3.346465,5.201403,9.166944,6.192108,3.850187,7.960717,8.586406,8.729021,13.567499,...,47.680974,23.840776,49.293623,53.167952,30.650253,33.059270,68.353950,70830.75,1.673402,149.0006
396,7.801407,2.628427,5.543288,9.133362,7.014197,5.730589,4.246228,6.930445,20.505433,43.245446,...,50.628851,40.195478,29.783881,48.611503,24.333390,39.715531,29.428252,72485.31,1.614970,150.9995
398,7.094305,2.423077,2.934079,6.637124,4.526698,3.664447,4.901074,5.327160,17.190049,20.815255,...,19.473850,16.587847,22.185683,24.114444,17.959727,19.521095,26.108803,73446.17,0.853824,150.9995


In [61]:
(np.array([1798.5480936491524, 1674.2732111008672, 1794.2786856643647]) + np.array([3868.141600677256, 1899.4293720457517, 2055.44657284463]) + np.array([2198.368013197518, 1577.0980127111864, 1648.1305938586702]) + np.array([1815.417729777189, 2056.5256021326427, 1815.019990798251]))/4

array([2420.11885933, 1801.8315495 , 1828.21896079])